<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/Agregacao_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=87c6c3a8d6dedf57863dc953861c879ba09939ba3a47928bfdb4a2e818809eaf
  Stored in directory: /root/.cache/pip/wheels/67/c2/7c/a53325365fba358ffff35af84a2e14cf88c18052f88acfa5f0
Successfully built spark


In [2]:
# Importar as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean, max as max_, variance, min as min_, first, last, avg, round, row_number
from pyspark.sql.window import Window


In [3]:
from google.colab import files
uploaded = files.upload()

Saving videos-preparados.snappy.parquet to videos-preparados.snappy.parquet


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 2️ Criar a SparkSession
spark = SparkSession.builder.getOrCreate()


In [6]:
# 3️ Ler o arquivo
df_video = spark.read.parquet('videos-preparados.snappy.parquet')
df_video.show(5)

+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|Month|Keyword Index|        Features PCA|     Features Normal|            Features|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|ASMR MUKBANG DOUB...|--ZI0dSbbNU|  2020-04-18|mukbang|378858|   18860|17975269|   18372987|2020|    4|         30.0|[0.6985786560867407]|[0.02303716158264...|[378858.0,1.79752...|
|Deadly car bomb d...|--hxd1CrOqg|  2022-08-22|   news|  6379|    4853|  808787|     820019|2022|    8|         37.0|[0.8936407990235931]|[3.87946679100418...|[6379.0,808787.0,...|
|How Biden&#39;s s...|--ixiTypG8g|  2022-08-24|   news|  1029|    2347|   97434|     100810|202

In [7]:
# 4 Contar a quantidade de registros por 'Palavra-chave' (Keyword)
df_video.groupBy('Keyword').agg(count('*').alias('Qtd Registros')).show()

+----------------+-------------+
|         Keyword|Qtd Registros|
+----------------+-------------+
|computer science|           48|
|            lofi|           40|
|         finance|           39|
|             cnn|           50|
|           apple|           42|
|            news|           39|
|         mukbang|           45|
|       education|           24|
|       interview|           50|
|          crypto|           50|
|   mathchemistry|           15|
|            food|           48|
|    data science|           50|
|        trolling|           50|
|        tutorial|           50|
|      literature|           46|
|             sat|           49|
|         history|           49|
|           cubes|           49|
|           music|           46|
+----------------+-------------+
only showing top 20 rows



In [8]:
# 5️ Calcular a média da coluna 'Interação' por 'Palavra-chave'
df_video.groupBy('Keyword').agg(mean('Interaction').alias('Média Interação')).show()

+----------------+--------------------+
|         Keyword|     Média Interação|
+----------------+--------------------+
|computer science|  1226793.0208333333|
|            lofi|         4167085.875|
|         finance|   708542.9487179487|
|             cnn|           570650.86|
|           apple|1.0873628214285715E7|
|            news|  251688.71794871794|
|         mukbang|1.1053630377777778E7|
|       education|         2750838.625|
|       interview|          3044867.04|
|          crypto|            413676.2|
|   mathchemistry|  3427342.7333333334|
|            food|   5352944.104166667|
|    data science|           562465.28|
|        trolling|          1484584.88|
|        tutorial|           6936688.3|
|      literature|            881726.5|
|             sat|           1098927.0|
|         history| 1.565269257142857E7|
|           cubes|1.5043961224489795E7|
|           music|2.9691370304347824E7|
+----------------+--------------------+
only showing top 20 rows



In [9]:
# 6️ Calcular o valor máximo da coluna 'Interaction' por 'Keyword' e ordenar
df_video.groupBy('Keyword').agg(
    max_('Interaction').alias('Rank Interactions')
).orderBy(col('Rank Interactions').desc()).show()


+--------+-----------------+
| Keyword|Rank Interactions|
+--------+-----------------+
| animals|       1593623628|
|   music|        922551152|
|     bed|        532691631|
| history|        440187490|
|   apple|        429916936|
| mrbeast|        300397699|
|  google|        239385460|
|business|        210025196|
|   cubes|        170925917|
|  sports|        106924567|
| mukbang|         87433858|
|    lofi|         86445177|
|tutorial|         69616442|
|  movies|         65253870|
|  marvel|         56247330|
|  how-to|         53053975|
|    food|         48754479|
| physics|         43463298|
|    asmr|         34411125|
|nintendo|         32268486|
+--------+-----------------+
only showing top 20 rows



In [10]:
# 7️ Calcular média e variância da coluna 'Visualizações' por 'Palavra-chave'
df_video.groupBy('Keyword').agg(
    mean('Views').alias('Média Visualizações'),
    variance('Views').alias('Variância Visualizações')
).show()


+----------------+--------------------+-----------------------+
|         Keyword| Média Visualizações|Variância Visualizações|
+----------------+--------------------+-----------------------+
|computer science|  1191958.7083333333|    2.81219868165102E12|
|            lofi|           4089363.0|   1.846209641476677...|
|         finance|   694223.4358974359|   3.304483175097042...|
|             cnn|           554240.38|   1.563423618468118...|
|           apple|1.0746930452380951E7|   4.299927977442589E15|
|            news|   247492.1794871795|   1.067512576672564...|
|         mukbang|1.0904772355555555E7|   5.586073238973179...|
|       education|  2684432.8333333335|   1.833572249339214...|
|       interview|          2966111.86|   1.819220996034335E13|
|          crypto|           404608.22|   3.513691634369074E12|
|   mathchemistry|  3328125.2666666666|   2.491467065256849...|
|            food|          5252406.25|   7.326374128154842E13|
|    data science|           544771.98| 

In [11]:
# 8️ Calcular média, mínimo e máximo de 'Visualizações' por 'Palavra-chave' SEM casas decimais
df_video.groupBy('Keyword').agg(
    round(mean('Views'), 0).alias('Média Visualizações'),
    round(min_('Views'), 0).alias('Mínimo Visualizações'),
    round(max_('Views'), 0).alias('Máximo Visualizações')
).show()

+----------------+-------------------+--------------------+--------------------+
|         Keyword|Média Visualizações|Mínimo Visualizações|Máximo Visualizações|
+----------------+-------------------+--------------------+--------------------+
|computer science|          1191959.0|               16115|             7004107|
|            lofi|          4089363.0|                6817|            84747957|
|         finance|           694223.0|                1195|             9450554|
|             cnn|           554240.0|               51269|             1889320|
|           apple|         1.074693E7|                1954|           425478119|
|            news|           247492.0|               10642|             1465011|
|         mukbang|        1.0904772E7|                3618|            86169225|
|       education|          2684433.0|                6611|            17103736|
|       interview|          2966112.0|                2587|            22529756|
|          crypto|          

In [13]:
# 9️ Mostrar o primeiro e o último 'Published At' para cada 'Keyword'
df_video.groupBy('Keyword').agg(
    first('Published At').alias('Primeiro Published At'),
    last('Published At').alias('Último Published At')
).show(truncate=False)

+----------------+---------------------+-------------------+
|Keyword         |Primeiro Published At|Último Published At|
+----------------+---------------------+-------------------+
|computer science|2022-02-08           |2020-09-08         |
|lofi            |2022-06-07           |2020-07-19         |
|finance         |2020-09-23           |2017-12-31         |
|cnn             |2022-08-17           |2022-08-13         |
|apple           |2022-08-22           |2022-08-02         |
|news            |2022-08-22           |2022-08-23         |
|mukbang         |2020-04-18           |2022-08-24         |
|education       |2015-02-06           |2010-10-14         |
|interview       |2021-08-03           |2018-10-05         |
|crypto          |2022-08-23           |2022-08-22         |
|mathchemistry   |2020-08-11           |2019-10-04         |
|food            |2022-07-17           |2022-08-20         |
|data science    |2019-08-18           |2021-08-06         |
|trolling        |2022-0

In [14]:
# 10 Contar todos os títulos normalmente e contar os únicos e comparar
total_titulos = df_video.select('Title').count()
titulos_unicos = df_video.select('Title').distinct().count()

print(f'Total de Títulos: {total_titulos}')
print(f'Total de Títulos Únicos: {titulos_unicos}')
print(f'Diferença: {total_titulos - titulos_unicos}')

Total de Títulos: 1869
Total de Títulos Únicos: 1854
Diferença: 15


In [15]:
 # 11 Contar registros por ano, ordenado ascendente
df_video.groupBy('Year').agg(count('*').alias('Qtd Registros')).orderBy('Year').show()

# 2 Contar registros por ano e mês, ordenado ascendente
df_video.groupBy('Year', 'Month').agg(count('*').alias('Qtd Registros')).orderBy('Year', 'Month').show()

# 3️ Calcular a média acumulativa de 'Likes' por 'Keyword' ao longo dos anos
window_spec = Window.partitionBy('Keyword').orderBy('Year').rowsBetween(Window.unboundedPreceding, 0)

df_video.withColumn('Média Acumulada Likes', avg('Likes').over(window_spec)).select(
    'Keyword', 'Year', 'Likes', 'Média Acumulada Likes'
).orderBy('Keyword', 'Year').show

+----+-------------+
|Year|Qtd Registros|
+----+-------------+
|2007|            2|
|2008|            1|
|2009|            9|
|2010|            6|
|2011|            4|
|2012|           12|
|2013|            6|
|2014|           10|
|2015|           15|
|2016|           34|
|2017|           47|
|2018|           57|
|2019|           86|
|2020|          158|
|2021|          229|
|2022|         1193|
+----+-------------+

+----+-----+-------------+
|Year|Month|Qtd Registros|
+----+-----+-------------+
|2007|    7|            1|
|2007|   12|            1|
|2008|    7|            1|
|2009|    2|            2|
|2009|    6|            2|
|2009|    7|            1|
|2009|    8|            1|
|2009|   10|            1|
|2009|   12|            2|
|2010|    3|            1|
|2010|    5|            2|
|2010|    6|            1|
|2010|    9|            1|
|2010|   10|            1|
|2011|    2|            1|
|2011|    5|            1|
|2011|    9|            1|
|2011|   10|            1|
|2012|    1|

<bound method DataFrame.show of DataFrame[Keyword: string, Year: int, Likes: int, Média Acumulada Likes: double]>

In [16]:
spark.stop()